In [ ]:
from astroquery.simbad import Simbad
import pandas as pd
from time import sleep

def get_messier_objects():
    """
    Retrieves details for all 110 Messier objects using astroquery's Simbad interface.
    Returns a pandas DataFrame with the collected information.
    """
    # Configure Simbad to return additional fields
    custom_simbad = Simbad()
    votable_fields = Simbad.list_votable_fields()
    for field in votable_fields:
       print(field['name'  ] , "\t" , field['description'] )
    custom_simbad.add_votable_fields('alltypes')
    
    # Initialize lists to store results
    results = []
    
    # Loop through all Messier objects (M1 to M110)
    for i in range(1, 111):
        messier_id = f"M {i}"
        try:
            # Query Simbad database
            result_table = custom_simbad.query_object(messier_id)
            
            if result_table is not None:
                # Extract data from the result
                data = {
                    'Messier': messier_id,
                    'Name': result_table['MAIN_ID'][0].decode(),
                    'RA': result_table['RA'][0],
                    'DEC': result_table['DEC'][0],
                    'Type': result_table['OTYPE'][0].decode() if result_table['OTYPE'][0] is not None else 'Unknown',
                    'Morphology': result_table['MORPHTYPE'][0].decode() if result_table['MORPHTYPE'][0] is not None else 'Unknown',
                    'Dimensions': result_table['DIM'][0].decode() if result_table['DIM'][0] is not None else 'Unknown',
                    'Distance': result_table['Distance_distance'][0] if result_table['Distance_distance'][0] is not None else None,
                    'Magnitude': result_table['MAG'][0] if result_table['MAG'][0] is not None else None
                }
                results.append(data)
                print(f"Successfully retrieved data for {messier_id}")
            else:
                print(f"No data found for {messier_id}")
                
            # Add a small delay to avoid overwhelming the server
            sleep(1)
            
        except Exception as e:
            print(f"Error retrieving data for {messier_id}: {str(e)}")
            continue
    
    # Convert results to DataFrame
    df = pd.DataFrame(results)
    
    # Clean up the data
    if 'Distance' in df.columns:
        df['Distance'] = pd.to_numeric(df['Distance'], errors='coerce')
    if 'Magnitude' in df.columns:
        df['Magnitude'] = pd.to_numeric(df['Magnitude'], errors='coerce')
    
    return df

def save_results(df, csv_path='messier_objects.csv'):
    """
    Saves the DataFrame to a CSV file.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing Messier object data
    csv_path (str): Path where the CSV file should be saved
    """
    df.to_csv(csv_path, index=False)
    print(f"Results saved to {csv_path}")

def main():
    print("Starting Messier object data collection...")
    df = get_messier_objects()
    
    # Display basic statistics
    print("\nData collection complete!")
    print(f"Total objects retrieved: {len(df)}")
    
    # Save the results
    save_results(df)
    
    # Display some example data
    print("\nSample of collected data:")
    print(df.head())
    
    # Display some basic statistics
    print("\nBasic statistics for numerical values:")
    print(df[['Distance', 'Magnitude']].describe())

 
get_messier_objects()

mesDiameter 	 Collection of stellar diameters.
mesPM 	 Collection of proper motions.
mesISO 	 Infrared Space Observatory (ISO) observing log.
mesSpT 	 Collection of spectral types.
allfluxes 	 all flux/magnitudes U,B,V,I,J,H,K,u_,g_,r_,i_,z_
ident 	 Identifiers of an astronomical object
flux 	 Magnitude/Flux information about an astronomical object
mesOtype 	 Other object types associated with an object with origins
mesPLX 	 Collection of trigonometric parallaxes.
otypedef 	 all names and definitions for the object types
mesDistance 	 Collection of distances (pc, kpc or Mpc) by several means.
otypes 	 List of all object types associated with an object
mesVar 	 Collection of stellar variability types and periods.
mesXmm 	 XMM observing log.
mesVelocities 	 Collection of HRV, Vlsr, cz and redshifts.
has_ref 	 Associations between astronomical objects and their bibliographic references
mesRot 	 Stellar Rotational Velocities.
biblio 	 Bibliography
ids 	 all names concatenated with pipe
mes

In [ ]:
import matplotlib.pyplot as plt
from astroquery.simbad import Simbad
import numpy as np

# Configure Simbad to return the necessary fields
Simbad.add_votable_fields('distance_result', 'flux(V)')

# Query Simbad for Messier objects
result_table = Simbad.query_criteria('cat=M')

# Filter out objects without distance or magnitude information
filtered_table = result_table[result_table['distance_result'].mask == False]
filtered_table = filtered_table[filtered_table['FLUX_V'].mask == False]

# Sort by distance and select the 10 nearest objects
sorted_table = filtered_table[np.argsort(f iltered_table['distance_result'])]
nearest_10 = sorted_table[:10]

# Extract distances and magnitudes
distances = nearest_10['distance_result']
magnitudes = nearest_10['FLUX_V']

# Plot distance vs magnitude
plt.figure(figsize=(10, 6))
plt.scatter(distances, magnitudes, color='blue')
plt.title('Distance vs Magnitude for the 10 Nearest Messier Objects')
plt.xlabel('Distance (pc)')
plt.ylabel('Magnitude (V)')
plt.gca().invert_yaxis()  # Magnitudes are plotted with brighter objects at the top
plt.grid(True)
plt.show()